# Sentiment Analysis

In [ ]:
%pip install pandas numpy scikit-learn matplotlib datasets transformers ipywidgets ipykernel

In [ ]:
import pandas as pd
import numpy as np
import sklearn as sklearn
from datasets import load_dataset
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV


# Mostly for the data preprocessing
def preprocess(text):
    text = text.lower()
    text = re.sub(r'https?://S+|www\.S+', '', text) #This removes URLs
    text = re.sub(r'\S*@\S*\s?', '', text)  # Remove emails
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

In [ ]:
# Breaking the existing dataset into 3 sectors: Train, Validation, Tests
data = load_dataset('go_emotions')
df_train = pd.DataFrame(data["train"])
df_val = pd.DataFrame(data["validation"])
df_test = pd.DataFrame(data["test"])

In [ ]:
# Next, we wanna use vectorization to convert the text data into numerical data.. Since ML algos dont understand 
vectorizer = TfidfVectorizer(max_features=100000, ngram_range=(1,2))  # Bigram TF-IDF
X_train_tfidf = vectorizer.fit_transform(df_train["text"])
X_val_tfidf = vectorizer.transform(df_val["text"])
X_test_tfidf = vectorizer.transform(df_test["text"])


In [ ]:
# Now we wanna convert the labels into a multi-label format
from sklearn.preprocessing import MultiLabelBinarizer

# Convert lists into multi-label format
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(df_train["labels"])
y_val = mlb.transform(df_val["labels"])
y_test = mlb.transform(df_test["labels"])

In [ ]:
param_grid = {
    "n_estimators": [300, 500, 700],
    "max_depth": [20, 30, 40],
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring="f1_weighted")
grid_search.fit(X_train_tfidf, y_train)

print("Best Params:", grid_search.best_params_)


In [ ]:
# Next, we wanna split the data into training and Validation sets.. Now this is really important since we wanna make sure that everything is gonna work perfecty  
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42)

In [ ]:
# Now we wanna train the model.. We are gonna use the Random Forest Classifier
# Initialize Random Forest Model
rf_model = RandomForestClassifier(n_estimators=500, class_weight="balanced", random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

In [ ]:
# Checking the model performance  
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="weighted")
print(f"Model Accuracy: {accuracy:.4f}")
print(f"Model F1 Score: {f1:.4f}")